In [1]:
import collections
import math
import os
import random
import zipfile
import datetime as dt
import numpy as np
import tensorflow as tf

In [2]:
# 输入训练路径
train_path = '/home/jiang/data/sorted10000'
# 基站路径
station_path = '/home/jiang/data/stationRecord'
# 输入长度
maxlen = 150
# 输入中某地（如海口）大于该值，进入训练集
threshold = 120
# 滚动步长
step = 1
# 测试长度
testlen = 50
batch_size=16
num_skips=2
skip_window=2
data_index1=0
data_index2=0
station_size=200
lr=0.01

valid_size = 16     # Random set of words to evaluate similarity on.
valid_window = 100  # Only pick dev samples in the head of the distribution.
valid_examples = np.random.choice(valid_window, valid_size, replace=False)
num_sampled = 64    # Number of negative examples to sample.

num_steps = 2000

station_size = 10000
embedding_size = 100

In [3]:
def read_data(train_path):
    file_object = open(train_path)
    train_list = file_object.readlines()
    train_set=[]
    user_record=[]
    station_list=[]
    station_set=set()
    for i,record in enumerate(train_list):
        record=record.strip()
        user,station=(record.split(",")[j] for j in [0,3])
        station_set.add(int(station))
        if i == len(train_list)-1:
            user_record.append(int(station))
            train_set.append(user_record)
            station_list=sorted(list(station_set))
            return train_set,station_list
        elif user != train_list[i+1].split(",")[0]:
            user_record.append(int(station))
            train_set.append(user_record)
            user_record=[]
        else:
            user_record.append(int(station))
    #return train_set,station_list

In [4]:
#注意不要让序列小于3个的序列进入
def generate_batch(data, batch_size, num_skips, skip_window):
    global data_index1
    global data_index2
    assert batch_size % num_skips == 0
    assert num_skips <= 2 * skip_window
    batch = np.ndarray(shape=(batch_size), dtype=np.int32)
    context = np.ndarray(shape=(batch_size, 1), dtype=np.int32)
    span = 2 * skip_window + 1  # [ skip_window input_word skip_window ]
    buffer = collections.deque(maxlen=span)
    #print len(data[data_index1])
    for i in range(batch_size // num_skips):
        #第一次进入循环初始化buffer
        if i == 0:
            while len(data[data_index1])-data_index2 < span:
                data_index2 = 0
                data_index1 = (data_index1 + 1)%len(data)                
            for _ in range(span-1):
                buffer.append(data[data_index1][data_index2])
                data_index2 += 1
        ###############################
        #index更新后判断是否越界
        if data_index2 == (len(data[data_index1])):
            data_index2 = 0
            data_index1 = (data_index1 + 1)%len(data)
            while len(data[data_index1])-data_index2 < span:
                data_index1 = (data_index1 + 1)%len(data)
            for _ in range(span-1):
                buffer.append(data[data_index1][data_index2])
                data_index2 += 1
        #################################
        #print data_index1,data_index2 
        buffer.append(data[data_index1][data_index2])
        #print buffer
        data_index2 += 1
        target = skip_window  # input word at the center of the buffer
        targets_to_avoid = [skip_window]
        for j in range(num_skips):
            while target in targets_to_avoid:
                target = random.randint(0, span - 1)
            targets_to_avoid.append(target)
            batch[i * num_skips + j] = buffer[skip_window]  # this is the input word
            context[i * num_skips + j, 0] = buffer[target]  # these are the context words
        
    # Backtrack a little bit to avoid skipping words in the end of a batch
    #data_index2 = (data_index2 + len(data) - span) % len(data)
    return batch, context

In [5]:
def recount_data(data,station_indices):
    new_data=[]
    for record in data:
        new_record=[]
        for station in record:
            new_record.append(station_indices[station])
        new_data.append(new_record)
    return new_data

In [6]:
def data_pre(train_path):
    train_data,station_data=read_data(train_path)
    sum_station = len(station_data)
    station_indices = dict((c, i) for i, c in enumerate(station_data))
    indices_station = dict((i, c) for i, c in enumerate(station_data))
    train_data = recount_data(train_data,station_indices)
    return train_data,sum_station,indices_station

In [7]:
data,station_num,indices_station=data_pre(train_path)
print station_num

1183


In [8]:
graph = tf.Graph()

with graph.as_default():

  # Input data.
    train_inputs = tf.placeholder(tf.int32, shape=[batch_size])
    train_context = tf.placeholder(tf.int32, shape=[batch_size, 1])
    valid_dataset = tf.constant(valid_examples, dtype=tf.int32)

  # Look up embeddings for inputs.
    embeddings = tf.Variable(
      tf.random_uniform([station_num, embedding_size], -1.0, 1.0))
    embed = tf.nn.embedding_lookup(embeddings, train_inputs)
    
    
    # Construct the variables for the NCE loss
    nce_weights = tf.Variable(
        tf.truncated_normal([station_num, embedding_size],
                            stddev=1.0 / math.sqrt(embedding_size)))
    nce_biases = tf.Variable(tf.zeros([station_num]))

    nce_loss = tf.reduce_mean(
        tf.nn.nce_loss(weights=nce_weights,
                       biases=nce_biases,
                       labels=train_context,
                       inputs=embed,
                       num_sampled=num_sampled,
                       num_classes=station_num))

    optimizer = tf.train.GradientDescentOptimizer(lr).minimize(nce_loss)
    
#   # Construct the variables for the softmax
#     weights = tf.Variable(tf.truncated_normal([embedding_size, station_num],
#                                               stddev=1.0 / math.sqrt(embedding_size)))
#     biases = tf.Variable(tf.zeros([station_num]))
#     hidden_out = tf.transpose(tf.matmul(tf.transpose(weights), tf.transpose(embed))) + biases

#   # convert train_context to a one-hot format
#     train_one_hot = tf.one_hot(train_context, station_num)

#     cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=hidden_out, labels=train_one_hot))

#   # Construct the SGD optimizer using a learning rate of lr.
#     optimizer = tf.train.GradientDescentOptimizer(lr).minimize(cross_entropy)

  # Compute the cosine similarity between minibatch examples and all embeddings.
    norm = tf.sqrt(tf.reduce_sum(tf.square(embeddings), 1, keep_dims=True))
    normalized_embeddings = embeddings / norm
    valid_embeddings = tf.nn.embedding_lookup(normalized_embeddings, valid_dataset)
    similarity = tf.matmul(valid_embeddings, normalized_embeddings, transpose_b=True)
    
    #只保存embeddings
    #saver = tf.train.Saver({"station_embeddings": embeddings})
    #保存全部变量
    saver = tf.train.Saver()


  # Add variable initializer.
    init = tf.global_variables_initializer()

In [13]:
session=tf.Session(graph=graph)
average_loss = 0
softmax_start_time = dt.datetime.now()
#重新训练使用session.run(init)
#session.run(init)
#载入之前的训练结果用saver
saver.restore(session, "./data/station_embedding.ckpt")
for step in range(num_steps):
    batch_inputs, batch_context = generate_batch(data,batch_size, num_skips, skip_window)
    #print batch_inputs,batch_context
    feed_dict = {train_inputs: batch_inputs, train_context: batch_context}
    #print feed_dict
    _, loss_val = session.run([optimizer, nce_loss], feed_dict=feed_dict)
    #_, loss_val = session.run([optimizer, cross_entropy], feed_dict=feed_dict)
    average_loss += loss_val
    #print step
    if step % 10 == 0:
        if step > 0:
            average_loss /= 10
    # The average loss is an estimate of the loss over the last 2000 batches.
            print('Average loss at step ', step, ': ', average_loss)
            average_loss = 0

    # Note that this is expensive (~20% slowdown if computed every 500 steps)
    if step % 200 == 0:
        sim = similarity.eval(session=session)
        for i in range(valid_size):
            valid_station = indices_station[valid_examples[i]]
            top_k = 8  # number of nearest neighbors
            nearest = (-sim[i, :]).argsort()[1:top_k + 1]
            log_str = 'Nearest to %s:' % valid_station
            for k in range(top_k):
                close_station = indices_station[nearest[k]]
                log_str = '%s %s,' % (log_str, close_station)
            print(log_str)
            
save_path = saver.save(session, "./data/station_embedding.ckpt")
    
#final_embeddings = normalized_embeddings.eval(session=session)
softmax_end_time = dt.datetime.now()
print("Softmax method took {} secs to run".format((softmax_end_time-softmax_start_time).total_seconds()))

INFO:tensorflow:Restoring parameters from ./data/station_embedding.ckpt
Nearest to 10773: 32113, 10331, 10311, 21512, 15681, 42933, 43252, 44041,
Nearest to 10603: 43362, 47312, 15391, 19811, 21953, 47261, 50111, 11081,
Nearest to 10782: 30854, 43831, 43431, 12903, 12113, 11962, 11342, 42152,
Nearest to 10701: 15192, 43732, 13881, 50561, 24333, 44501, 44011, 10747,
Nearest to 10472: 19943, 43512, 19071, 49222, 20062, 42313, 18101, 39111,
Nearest to 10802: 10271, 43482, 10811, 13231, 31181, 13713, 14813, 10141,
Nearest to 10383: 43861, 19114, 13713, 50021, 31262, 12793, 30292, 13233,
Nearest to 10532: 13133, 42201, 17012, 42292, 42501, 43741, 11393, 43373,
Nearest to 10671: 13273, 42493, 47201, 50243, 42202, 44091, 13353, 42321,
Nearest to 10271: 13713, 13021, 14813, 38127, 30292, 42033, 13122, 50021,
Nearest to 10351: 22106, 43953, 14011, 13571, 10641, 30901, 11262, 42561,
Nearest to 10401: 14122, 43253, 12061, 18566, 50243, 15392, 20323, 30663,
Nearest to 10181: 11762, 42521, 43483, 1

('Average loss at step ', 690, ': ', 13.646129512786866)
('Average loss at step ', 700, ': ', 8.2581789493560791)
('Average loss at step ', 710, ': ', 15.784983491897583)
('Average loss at step ', 720, ': ', 23.372243976593019)
('Average loss at step ', 730, ': ', 25.353354358673094)
('Average loss at step ', 740, ': ', 20.283594322204589)
('Average loss at step ', 750, ': ', 15.153756856918335)
('Average loss at step ', 760, ': ', 21.870773124694825)
('Average loss at step ', 770, ': ', 23.121040630340577)
('Average loss at step ', 780, ': ', 21.684462070465088)
('Average loss at step ', 790, ': ', 18.823307895660399)
('Average loss at step ', 800, ': ', 25.750524139404298)
Nearest to 10773: 32113, 10331, 21512, 10311, 15681, 42933, 43252, 44041,
Nearest to 10603: 43362, 47312, 19811, 15391, 21953, 47261, 11081, 50111,
Nearest to 10782: 30854, 43831, 43431, 12903, 12113, 11962, 11342, 42152,
Nearest to 10701: 15192, 43732, 13881, 50561, 24333, 44501, 44011, 10747,
Nearest to 10472: 19

('Average loss at step ', 1440, ': ', 13.790077114105225)
('Average loss at step ', 1450, ': ', 0.74666515290737157)
('Average loss at step ', 1460, ': ', 1.1297217816114427)
('Average loss at step ', 1470, ': ', 5.8842518404126167)
('Average loss at step ', 1480, ': ', 13.306545245647431)
('Average loss at step ', 1490, ': ', 5.8775837957859043)
('Average loss at step ', 1500, ': ', 15.027419805526733)
('Average loss at step ', 1510, ': ', 7.5532803058624269)
('Average loss at step ', 1520, ': ', 11.808721613883971)
('Average loss at step ', 1530, ': ', 6.6241594791412357)
('Average loss at step ', 1540, ': ', 15.436310768127441)
('Average loss at step ', 1550, ': ', 22.326939392089844)
('Average loss at step ', 1560, ': ', 24.768576145172119)
('Average loss at step ', 1570, ': ', 14.073309183120728)
('Average loss at step ', 1580, ': ', 16.263311862945557)
('Average loss at step ', 1590, ': ', 20.292783641815184)
('Average loss at step ', 1600, ': ', 20.485971641540527)
Nearest to 10